## Part 0. General settings

This report is written in Jupyter Notebook, and all SQL queries are exectuted as python script. Therefore, it follows the sequence of code executing, and all codes share the same namespace. 

The database server is set up on Amazon Web Service (AWS). The address and other related information can be found below.

Here are the general scripts used through the whole project:

In [4]:
"""Display settings"""
from IPython.display import HTML, display
import tabulate

# optional `heading` arg. If provided it will be added to the first row as the table heading.
def displayResult(queryResult, heading=()):
    if heading != ():
        resultList = (heading,) + queryResult
        display(HTML(tabulate.tabulate([result for result in resultList], tablefmt='html')))
    else:
        display(HTML(tabulate.tabulate([result for result in queryResult], tablefmt='html')))

"""MySQL connection related functions and variables"""

import pymysql

def open_conn():
    """open the connection before each test case"""
    conn = pymysql.connect(user='public', password='ece656yelp',
                                   host='maindb.czbva1am4d4u.us-east-2.rds.amazonaws.com',
                                   database='yelp_db')
    return conn

def close_conn(conn):
    """close the connection after each test case"""
    conn.close()

def executeQuery(conn, query, commit=False, returnResult=True):
    """ fetch result after query"""
    cursor = conn.cursor()
    query_num = query.count(";")
    if query_num > 1:
        if not returnResult:
            cursor.execute(query)
            return
        else:
            for result in cursor.execute(query, params=None, multi=True):
                if result.with_rows:
                    result = result.fetchall()
    else:
        cursor.execute(query)
        result = cursor.fetchall()
    # we commit the results only if we want the updates to the database
    # to persist.
    if commit:
        conn.commit()
    else:
        conn.rollback()
    # close the cursor used to execute the query
    cursor.close()
    return result

yelp_conn = open_conn()

## Part I - 1.1 Database structure
Here is the Entity-Relationship diagram of the database. The attributes are not shown because there are too many for some of the tables. Instead, they can be find in the following figure illustrating the relational model.
![ER-Diagram](../../ER_Diagram.png)

Here is the relational model of the database. Primary keys are marked as orange and foreign keys are marked as blue. Also the arrows indicate the reference relations between the tables.
![R-Model](../../RelationalModel.png)

## Part I - 1.2 Data cleaning

First some sanity checks and consistency checks were performed on the yelp database. The data in the database should follow rules and should be consistent with each other. After studying the relationship models in of it, we perfomed the following checks and cleaning:
- Logical time checks: review or becoming an elite member can not occur before yelp was founded or from the future. 
- Logic consistency checks: a user can not leave a review before creating the account, a user can not become an elite member before becoming a yelp user, people should not check in at a business outside open hours, and the reviews written by each user should be in consistent with that in the user table.

We first got the queries ready, then came up with the following indices to accelerate the running of queries. As we are going to time the execution commands, they are added before further discussion, as shown in 1.2.1. After that in 1.2.2, we will introduce the checking queries and the measures taken to deal with the inconsistency and broken data. 

### 1.2.1 Indexing the sanity check queries

To allow for faster queries additional indices were added to some tables in the database. This includes:
* adding one to the year column in the table elite_years to speed up query_6
  * create index idx_year on elite_years(year)
* adding one to the yelping_since column in the user table
  * create index idx_yelping_since on user(yelping_since)
* creating new columns in the hours table to split the hours column into date, opening_time and closing_time and creating an index for each of these
  * create index idx_day_of_week on hours_new(day_of_week)
* creating new columns in the checkin table to split the date column into date_of_week and time and creating an index for each of these
  * create index idx_day_of_week on hours_new(day_of_week)
* creating new columns in the review table to split the date column into year, month, day columns and scrapping the time because its always 00:00:00
  * select count(*) from review where substring_index(date, ' ', -1) != '00:00:00'; //returns 0

In [6]:
index_1_1 = "create index idx_year on elite_years(year);"
index_1_2 = "create index idx_yelping_since on user(yelping_since);"

executeQuery(yelp_conn, index_1_1, returnResult=False)
executeQuery(yelp_conn, index_1_2, returnResult=False)

()

`index2` is too long to execute here. It uses a procedure to create a new `hour` table with openning and closing time as seperate columns, so we do not need to parse the time column to retrieve the openning and closing tome. In fact, we think it is a bad practice in the original database to put these two in the same columnn as it violates 1NF.
`index3` as shown below, creates indexes for this new table:

In [7]:
index_3_1 = "create index idx_day_of_week on hours_new(day_of_week);"
index_3_2 = "create index idx_opening_time on hours_new(opening_time);"
index_3_3 = "create index idx_closing_time on hours_new(closing_time);"
executeQuery(yelp_conn, index_3_1, returnResult=False)
executeQuery(yelp_conn, index_3_2, returnResult=False)
executeQuery(yelp_conn, index_3_3, returnResult=False)

()

Again, `index4` creates a new table based on checkin table which creates seperated columns for time digits.

This `index5` series, create the indexes for the new check_in table:

In [ ]:
index_5_1 =  "create index idx_day_of_week on checkin_new(day_of_week)"
index_5_2 =  "create index idx_opening_time on checkin_new(checkin_time)"
executeQuery(yelp_conn, query_5_1)
executeQuery(yelp_conn, query_5_2)

### 1.2.2 Sanity/consistency check and cleaning

#### 1. Check that no review is from the future or before Yelp's founding

In [24]:
query_1 = "SELECT id, date FROM review WHERE unix_timestamp(date) <= unix_timestamp('2004-10-01')\
           OR unix_timestamp(date) >= unix_timestamp('2018-01-01');"

%time result_1 = executeQuery(yelp_conn, query_1)

result_1

CPU times: user 1.85 ms, sys: 2.5 ms, total: 4.36 ms
Wall time: 2.75 s


(('03B9-gqbeGoMmPJbNzNT5w', datetime.datetime(2004, 9, 15, 0, 0)),
 ('PbIY2aIyszb6he6J-ey67w', datetime.datetime(2004, 7, 22, 0, 0)))

This shows 2 accounts that were created before Yelp's founding in October 2004.

The query runs in about 3 seconds as shown in the output. For the one without keys and indexes, result is shown below. Big improvement can be observed.

This query deletes them:

In [3]:
# clean_1 = "DELETE FROM review WHERE unix_timestamp(date) <= unix_timestamp('2004-10-01')\
#            OR unix_timestamp(date) >= unix_timestamp('2018-01-01');"

# executeQuery(yelp_conn, clean_1)

#### 2. Can't leave a review dated before account creation

In [8]:
query_2 =  "SELECT user.id, user.yelping_since, review.date\
            FROM (user INNER JOIN review ON user.id = review.user_id)\
            WHERE user.yelping_since > review.date\
            GROUP BY user.id;"

%time result_2 = executeQuery(yelp_conn, query_2)

CPU times: user 10.7 ms, sys: 2.18 ms, total: 12.9 ms
Wall time: 41.6 s


In [9]:
len(result_2)

196

This means these users somehow posted a review before their account was created, suggesting a glitch with their database. Here are 5 of users who have such illege review records:

In [10]:
heading_2 = ("user.id", "user.yelping_since", "review.date")
displayResult(result_2[:5], heading_2)

user.id,user.yelping_since,review.date
-58CWJ48is4duXgpvsWEGA,2013-09-18 00:00:00,2008-10-23 00:00:00
-9NfX8JO_5UVN_h1K8yOcg,2015-02-12 00:00:00,2010-02-26 00:00:00
-kEsfYKPs1_rgEWEIui2Mw,2015-02-14 00:00:00,2014-04-07 00:00:00
-KP8Me2KRqO7IwKIaFL-Vg,2013-10-19 00:00:00,2013-09-02 00:00:00
09T8OU8BDhQkiU8m4vZy_A,2013-10-21 00:00:00,2013-10-16 00:00:00


It takes about 40 seconds to run the query.

This query deletes illegal reviews:

In [7]:
# clean_2 =  "DELETE FROM review WHERE id in\ 
#             (SELECT review.id FROM (user INNER JOIN review ON user.id = review.user_id)\
#             WHERE user.yelping_since > review.date\
#             GROUP BY user.id);"

# executeQuery(yelp_conn, clean_2)

#### 3. Can't be elite in a year before their account was made

In [13]:
query_3 =  "SELECT user.id, user.yelping_since AS Date_of_yelping, elite_years.year AS Year_of_Elite\
            FROM (user INNER JOIN elite_years ON user.id = elite_years.user_id)\
            WHERE YEAR(user.yelping_since) > elite_years.year\
            GROUP BY user.id;"

%time result_3 = executeQuery(yelp_conn, query_3)

CPU times: user 993 µs, sys: 1.05 ms, total: 2.04 ms
Wall time: 728 ms


In [14]:
len(result_3)

0

No illegal elite users records are found in the notebook.

This query runs quite fast (within 1 second).

In [15]:
heading_3 = ("user.id", "user.yelping_since", "elite_year.year")
displayResult(result_3[:5], heading_3)

user.id,user.yelping_since,elite_year.year


#### 4. Can't checkin outside open hours

In [12]:
query_4 = "SELECT COUNT(*) FROM checkin_new AS c\
           JOIN (SELECT hours_new.business_id, day_of_week, opening_time, closing_time FROM hours_new)\
           AS a ON a.business_id = checkin_new.business_id\
           WHERE a.day_of_week = c.day_of_week\
           AND (a.opening_time > c.checkin_time\
           OR a.closing_time < c.checkin_time);"
%time result_4 = executeQuery(yelp_conn, query_4)
result_4[0]

KeyboardInterrupt: 

NameError: name 'result_4' is not defined

This shows that there are many check ins that occur outside of the businesses open hours which are potentially invalid checkins, but this is not a guarantee because it is possible that the business changed their hours of operation at some point after someone checked in, resulting in the discrepency. 

This also shows the need within the database for the date column in the checkin and the hours column in the hours table to be normalized by splitting into date, opening time and closing time columns as this would save computation time having to perform substring_index computations on every row. No change is made to the database to correct this because it may sometimes happen since the business may change its open hours without updating that on yelp.

#### 5. `User.review_count` cannot be less than the sum of the number of reviews by a user

In [16]:
query_5 = "select count(*) from user join (select count(user_id) as countedReviews, user_id from\
           review group by user_id) as a on a.user_id = user.id where a.countedReviews > review_count;"
%time result_5 = executeQuery(yelp_conn, query_5)
print(result_5[0])

CPU times: user 6.4 ms, sys: 4.47 ms, total: 10.9 ms
Wall time: 3min 49s
(1323,)


This shows that the way Yelp gets the review_count number is potentially flawed since it should never count there being less reviews than the number of reviews provided for each user, unless the dataset that it was acquiring the count from was out of date. 

The query runs in about 4 minutes.

This query updates the incorrect review_count:

In [ ]:
# clean_5_1 = "CREATE VIEW review_counts_for_users AS\
#              SELECT user_id, count(user_id) AS count\
#              FROM review GROUP BY user_id;"

# clean_5_2 = "UPDATE user SET review_count = (SELECT count\
#              FROM review_counts_for_users WHERE id=user_id);"

# executeQuery(yelp_conn, clean_5_1)
# executeQuery(yelp_conn, clean_5_2)

#### 6. Cannot be Elite in an invalid year
Invalid years include ones before 2004, years in the future or years they didn't post a review, tip or photo.

In [18]:
query_6 = "select count(*) from elite_years join \
           (SELECT user_id, SUBSTRING_INDEX(date, '-', 1) AS year FROM review) \
           as a on a.user_id=elite_years.user_id and a.year = elite_years.year \
           group by elite_years.user_id, elite_years.year;"
%time result_6 = executeQuery(yelp_conn, query_6)
print(result_6[0])

KeyboardInterrupt: 

NameError: name 'result_6' is not defined

In total there are 186900 entries in elite_years. 37 appear to be erroneous. This query deletes these incorrect elite records:

In [ ]:
# clean_6 = "DELETE FROM elite_years where user_id in 
#              (select E.user_id from (SELECT * FROM elite_years) AS E join\
#              (SELECT user_id, SUBSTRING_INDEX(date, '-', 1) AS year FROM review)\
#              as a on a.user_id=E.user_id and a.year = E.year\
#              group by E.user_id, E.year);"
# executeQuery(yelp_conn, clean_6)